# How close is "close"? An analysis of the spatial characteristics of perceived proximity using Large Language Models

> Shingleton, J., & Basiri, A. (2025). How close is “close”? An analysis of the spatial characteristics of perceived proximity using Large Language Models. AGILE GIScience Series, 6, 1–14. [10.1111/1755-2665.12276](https://doi.org/10.1111/1755-2665.12276)

In [ ]:
import pandas as pd
import math

from typing import List, Dict, Any,Tuple

# global grid
import h3 # Hexagonal grid indexing
from keplergl import KeplerGl  

# llm
import ollama

# json and text repair
import re
import json
from json_repair import repair_json
from rapidfuzz import fuzz

# geocode
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import osmnx as ox

# progress bar initialization
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
raw_airbnb_df = pd.read_csv("data/listings.csv").dropna(subset=["description"])

In [ ]:
## 3. Load and inspect the listings

use_cols = [ "id", "latitude", "longitude", "description", "neighborhood_overview", "neighbourhood_cleansed"]

airbnb_df = raw_airbnb_df[use_cols].copy()
airbnb_df = airbnb_df.dropna(subset=["latitude", "longitude"])
airbnb_df = airbnb_df.dropna(subset=["neighborhood_overview", "neighbourhood_cleansed"], how='all')

airbnb_df = airbnb_df.reset_index(drop=True)

airbnb_df.head()

In [ ]:
## 4. Spatial aggregation with H3

H3_RESOLUTION = 9

# extract h3 cell id from point
def latlon_to_h3(row: pd.Series, resolution: int = H3_RESOLUTION):
    """
    Take a shapely Point (lon/lat) and return the H3 index.
    H3 expects (lat, lon) = (y, x).
    """
    return h3.latlng_to_cell(row.latitude, row.longitude, resolution)

airbnb_df['h3_cell'] = airbnb_df.apply(latlon_to_h3, axis=1)


In [ ]:
# convert point to h3 cell
h3_airbnb_density = (
    airbnb_df
    .groupby("h3_cell")
    .size()
    .reset_index(name="airbnb_count")
)

m = KeplerGl(height=600)
m.add_data(data=h3_airbnb_density, name="h3_airbnb_density")
m

In [ ]:
## 5. Filter your data to only include the inner city

inner_airbnb_df = airbnb_df[airbnb_df['neighbourhood_cleansed'] == 'Innere Stadt']
inner_airbnb_df.describe()

In [ ]:
h3_inner_airbnb_density = (
    inner_airbnb_df
    .groupby("h3_cell")
    .size()
    .reset_index(name="airbnb_count")
)

m = KeplerGl(height=600)
m.add_data(data=h3_inner_airbnb_density, name="h3_inner_airbnb_density")
m

In [ ]:
## 6. Definition of the extraction prompt

SYSTEM_PROMPT = """
You are an Airbnb location extraction system for Vienna. 
Your task is to extract specific Points of Interest (POIs) that are mentioned as being NEAR the property.

### 1. WHAT TO EXTRACT (Positive Criteria)
You must only extract specific, physical landmarks that can be pinpointed on a map.
- Specific Landmarks (e.g., "Schönbrunn Palace", "St. Stephen's Cathedral")
- Named Squares/Streets (e.g., "Stephansplatz", "Mariahilfer Straße", "Florianigasse")
- Named Markets/Parks (e.g., "Naschmarkt", "Prater", "Schönbornpark")
- Specific Transport Hubs (e.g., "Westbahnhof", "U4 Pilgramgasse")

### 2. WHAT TO IGNORE (Negative Criteria)
- IGNORE Districts and Quarters: Do NOT extract "Neubau", "Favoriten", "1st District", "Freihausviertel", "Museumsquartier" (unless referring to the specific complex, usually assume it's a district).
- IGNORE Generic Amenities: Do NOT extract "supermarkets", "restaurants", "bars", "shops", "metro station" (if unnamed).
- IGNORE Property Location: If the text says "The apartment is located in [Place]," ignore [Place]. Only extract places described as being *near*, *around*, or *walking distance* from the apartment.

### 3. ADDRESS FORMATTING RULES
- If a full address is in the text, use it.
- If NO address is in the text, you MUST format the address as: "[Name], Vienna".
- This is required formatting, not "inventing."

### 4. OUTPUT FORMAT
Do not include markdown formatting or explanations.
Keys: "name", "address", "latitude" (default null), "longitude" (default null), "context".

### 5. FEW-SHOT EXAMPLES (Do not confuse these with your real task)

Input: "We are located in the heart of Neubau, close to the Zieglergasse station and a short walk to Mariahilfer Straße."
Output:
[
  {
    "name": "Zieglergasse station",
    "address": "Zieglergasse station, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "close to the Zieglergasse station"
  },
  {
    "name": "Mariahilfer Straße",
    "address": "Mariahilfer Straße, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "short walk to Mariahilfer Straße"
  }
]

Input: "Enjoy the many shops and cafes in the 7th district. The bus takes you to the City Center."
Output:
[]

Input: "The apartment is located in the 3rd district, right next to the Belvedere Palace and just a short walk from the Rennweg station."
Output:
[
  {
    "name": "Belvedere Palace",
    "address": "Belvedere Palace, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "right next to the Belvedere Palace"
  },
  {
    "name": "Rennweg station",
    "address": "Rennweg station, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "short walk from the Rennweg station"
  }
]
Input: "You are staying directly opposite the famous Hundertwasserhaus."
Output:
[
  {
    "name": "Hundertwasserhaus",
    "address": "Hundertwasserhaus, Vienna",
    "latitude": null,
    "longitude": null,
    "context": "directly opposite the famous Hundertwasserhaus"
  }
]

### END OF EXAMPLES
Now, analyze the following description and extract the locations based strictly on the text provided below.
"""

In [ ]:
## 7. Calling an LLM locally (Ollama)

MODEL_NAME = "qwen3:1.7b"

ollama.pull(MODEL_NAME)

# Create a client to call your model 
client = ollama.Client()

In [ ]:
def ollama_extract_nearby(text: str, show_log: bool = True) -> Tuple[str, str]:
    """Return the raw model text output for one listing."""

    # define what you send to the model, the `system`` defines how the model should react, the `user` is the text you want to analyze 
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": text},
    ]
    # you call the model here 
    resp = client.chat(
        model=MODEL_NAME,
        messages=messages,
        options={"temperature": 0}, 
        think=True
    )

    model_text = resp["message"]["content"]
    thinking = resp["message"].get("thinking", "...no thinking...")

    if show_log:
        # print the model's thinking process
        print("-----")
        print(text)
        print("\n thinking... \n")
        print("response",thinking)
        print("\n\n =>\n")
        print(model_text)
    
    # return only the content of the response without the thinking part 
    return model_text, thinking


In [ ]:
## 8. Parsing the LLM output robustly

def parse_json_array(content: str) -> List[Dict[str, Any]]:
    """
    Try to parse the model output as a JSON array of objects.
    Be tolerant of minor formatting issues.
    """
    # First, optimistic: treat the whole content as JSON
    try:
        data = json.loads(content)
        if isinstance(data, list):
            return data
        # If the model returned an object, you might decide how to handle it,
        # but for this task we expect a list.
    except json.JSONDecodeError:
        pass
    
    
    # Second, optimistic: treat the whole content as JSON
    try:
        
        data = repair_json(content)
        data = json.loads(data)
        if isinstance(data, list):
            return data
        # If the model returned an object, you might decide how to handle it,
        # but for this task we expect a list.
    except json.JSONDecodeError:
        pass

    # Second, try to extract the first JSON array from the text
    match = re.search(r"\[\s*\{.*?\}\s*\]", content, flags=re.DOTALL)
    if match:
        try:
            data = json.loads(match.group(0))
            if isinstance(data, list):
                return data
        except json.JSONDecodeError:
            pass
    

    # Fallback: nothing parseable
    return []

In [ ]:
## 9. Batch extraction on a small sample

def extract_for_row(row) -> pd.Series:

    out = pd.Series({
        "locations": [],
        "thinking": None
    })

    desc = row.get("description", "")
    neigh = row.get("neighborhood_overview", "")

    content = f"{desc} {neigh}".strip()
    # you can also clean the text
    content = content.replace("\n", " ") # remove new lines
    content = content.replace("<br />", " ") # remove html tags

    if not content:
        return out

    try:    
        locations, thinking = ollama_extract_nearby(content)
        out["locations"] = parse_json_array(locations)
        out["thinking"] = thinking
        return out 
    except Exception as e:
        print(f"Error extracting locations for row: {e}")
        return out


In [ ]:
# get a small sample of the dataframe
small_sample = airbnb_df.sample(10)

small_sample[["locations", "thinking"]] = small_sample.progress_apply(extract_for_row, axis=1)

# change to inner_airbnb_df to run all the listings in the inner city
# inner_airbnb_df[["locations", "thinking"]] = inner_airbnb_df.progress_apply(extract_for_row, axis=1)


In [ ]:
## 9. Reduce hallucinations with fuzzy presence checks

# open parquet from data folder if needed
inner_airbnb_df = pd.read_parquet("data/airbnb_nearby.parquet")

In [ ]:
THRESH = 70

def filter_dicts_by_presence(row):
    # 1. Prepare the text to search in

    full_text = f"{row['description']} {row['neighborhood_overview']}".lower()
    
    kept_dicts = []
    
    # 2. Iterate through the existing list
    metadata_list = row['locations']
    print(metadata_list)

    for item in metadata_list:
        name = item.get('name', '')
        print(name)
        if not name:
            continue
            
        # 3. Keep the dict only if the name is found in the text
        # partial_ratio > 85 ensures it's a strong match (handles minor typos)
        if fuzz.partial_ratio(name.lower(), full_text) >= THRESH:
            print("Found")
            kept_dicts.append(item)
    print("-"*10)
    return kept_dicts

inner_airbnb_df['cleaned_metadata_list'] = inner_airbnb_df.progress_apply(filter_dicts_by_presence, axis=1)

In [ ]:
## 10. Adding missing coordinates to the cleaned locations

# 1. Initialize Nominatim
# IMPORTANT: You MUST provide a custom user_agent to identify your app
geolocator = Nominatim(user_agent="my_academic_project_graz_university")

# 2. Create a rate-limited version of the geocode function
# This automatically waits 1 second between calls to avoid banning
geocode_safe = RateLimiter(geolocator.geocode, min_delay_seconds=1.1)

# 3. Define a function to process your list of dicts
def geocode_locations(metadata_list):
    if not isinstance(metadata_list, list):
        return metadata_list
    
    for location in metadata_list:
        address = location.get("address")
        print(address)
        if address:
            try:
                # Call the safe geocoder
                loc = geocode_safe(address)
                print(loc)
                print(loc.latitude)
                print(loc.longitude)
                print("-" * 10)
                if loc:
                    location["latitude"] = loc.latitude
                    location["longitude"] = loc.longitude
                else:
                    location["latitude"] = None
                    location["longitude"] = None
                    # print(f"No location found for: {address}") # Optional logging
            except Exception as e:
                print(f"Error geocoding {address}: {e}")
                location["latitude"] = None
                location["longitude"] = None
    return metadata_list

# 4. Apply with progress bar & run the process
tqdm.pandas()
inner_airbnb_df['cleaned_metadata_list'] = inner_airbnb_df['cleaned_metadata_list'].progress_apply(geocode_locations)

In [ ]:
# open parquet from data folder if needed
inner_airbnb_df = pd.read_parquet("data/airbnb_nearby_with_coordinates.parquet")
list(inner_airbnb_df.locations)

In [ ]:
# get vienna geometry 
PLACE_NAME = "Vienna, Austria"

# Get the boundary polygon of Vienna and project it to EPSG:31256
poly_border_vienna = ox.geocode_to_gdf(PLACE_NAME)
poly_border_vienna.plot()

vienna_polygon = poly_border_vienna.geometry.iloc[0]

In [ ]:
# clean the coordinates if not in vienna or null

def clean_coordinates(row, geometry):
    # get bbox of vienna from the geometry
    min_lon, min_lat, max_lon, max_lat = geometry.bounds

    kept_dicts = [] # init a dict for returning the filtered values 
    for item in row['locations']:
        lon, lat = item.get('longitude'), item.get('latitude')

        # missing coordinates
        if lon is None or lat is None:
            continue

        # coordinates outside vienna
        if not (min_lon <= lon <= max_lon and min_lat <= lat <= max_lat):
            continue
        
        kept_dicts.append(item)
            
    return kept_dicts

# Apply to your column
inner_airbnb_df['cleaned_locations'] = inner_airbnb_df.apply(lambda row: clean_coordinates(row, vienna_polygon), axis=1)

In [ ]:
# count addresses
inner_airbnb_df['location_count'] = inner_airbnb_df['locations'].apply(lambda x: len(x))
inner_airbnb_df['cleaned_location_count'] = inner_airbnb_df['cleaned_locations'].apply(lambda x: len(x))
inner_airbnb_df["dropped_addresses"] = inner_airbnb_df['location_count'] - inner_airbnb_df['cleaned_location_count']

h3_adr_dropped = (
    inner_airbnb_df
    .groupby("h3_cell")['dropped_addresses']
    .sum()
    .reset_index(name="dropped_adr")
)

m = KeplerGl(height=600)
m.add_data(data=h3_adr_dropped, name="h3_adr_dropped")
m

In [ ]:
## 11. Reference locations and “area of influence” (standard distance)

airbnb_df_exploded = inner_airbnb_df.explode("cleaned_locations", ignore_index=True).dropna(subset=["cleaned_locations"]).reset_index(drop=True)
airbnb_df_exploded["loc_name"] = airbnb_df_exploded.cleaned_locations.str["name"]
airbnb_df_exploded["loc_lat"] = airbnb_df_exploded.cleaned_locations.str["latitude"]
airbnb_df_exploded["loc_lon"] = airbnb_df_exploded.cleaned_locations.str["longitude"]
airbnb_df_exploded.head(5)

In [ ]:
# change display options to show all rows
pd.set_option('display.max_rows', None)

airbnb_df_exploded.loc_name.value_counts()


In [ ]:
# reset count option
pd.set_option('display.max_rows', 10)

In [ ]:
value_counts = airbnb_df_exploded.loc_name.value_counts().head(25)
fig = px.histogram(x=value_counts.index, 
    y=value_counts.values,
    labels={'x': 'Location Name', 'y': 'Count'}, height=400)
fig

In [ ]:
# distance haversine

EARTH_RADIUS_KM = 6371.0088

def haversine_km(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """Great-circle distance (km) between two WGS84 points."""
    # TODO (fill in): implement haversine
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return EARTH_RADIUS_KM * c

In [ ]:
# get distance
airbnb_df_exploded = airbnb_df_exploded.dropna(subset=['longitude', 'latitude', 'loc_lon', 'loc_lat'])
airbnb_df_exploded['distance'] = airbnb_df_exploded.apply(lambda row: haversine_km(row['longitude'],row['latitude'], row['loc_lon'], row['loc_lat']), axis=1)

In [ ]:
h3_airbnb_density = (
    airbnb_df_exploded
    .groupby("h3_cell")['distance']
    .mean()
    .reset_index(name="avg_distance")
)

m = KeplerGl(height=600)
m.add_data(data=h3_airbnb_density, name="avg_distance")
m

**Congratulations! 🎉 You have successfully completed the exercise.**

In [ ]:
# this line is to clear the output of the notebook, so that when you commit it, it is clean
!jupyter nbconvert --clear-output --inplace crime_ex.ipynb